In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/iterative-tensor-decomposition-colab

/content/drive/MyDrive/iterative-tensor-decomposition-colab


In [3]:
from run.run_tca import decompose
import numpy as np
import torch
from core.utilities import get_filename
import pickle


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cbl = np.load('/content/drive/MyDrive/iterative-tensor-decomposition-colab/data/wagner_cbl_warped.npy')
ctx = np.load('/content/drive/MyDrive/iterative-tensor-decomposition-colab/data/wagner_ctx_warped.npy')

data = np.concatenate((cbl,ctx), axis=0)

data = torch.from_numpy(data).float().permute(1, 0, 2).to(device)

In [4]:
data.shape

torch.Size([218, 286, 150])

In [5]:
#The hyperparameters
slice_TCA = True
positive = False
orthogonal_constraint  = False
cross_validation = False
orthogonal_penalty = False
metric = False
cut_cv_mask = 8 # 8 samples = 0.26 s

mask_cross_validation = 0.2

learning_rate = 0.02
mask = 0.8

decay_rate_lr = 1
decay_rate_mask = 0.5
decay_iterations = 5
decay_type_lr='exponential'
decay_type_mask='exponential'

iterations = decay_iterations*199
batch_size = 20
test_freq = -1
initialization = 'uniform'

verbose_train = True
verbose_test = True

In [6]:
mask_cross_validation = [0]

res = []
for seed in range(10):
    res.append(decompose(seed=seed, data=data, number_components=[3,3,0], iterations=iterations,
                decay_rate_mask=decay_rate_mask, decay_rate_lr=decay_rate_lr, decay_iterations=decay_iterations,
                decay_type_mask=decay_type_mask, decay_type_lr=decay_type_lr, learning_rate=learning_rate, mask=mask,
                mask_cross_validation=mask_cross_validation[0], batch_size=batch_size, sliceTCA=slice_TCA, positive=positive,
                orthogonal_penalty=orthogonal_penalty, metric=metric, cross_validation=False,
                precision=torch.float32, orthogonal_constraint=orthogonal_constraint, cut_cv_mask=cut_cv_mask,
                initialization=initialization, orthogonal_skip=(), test_freq=-1, verbose_train=verbose_train,
                verbose_test=verbose_test, device=device))

Iteration: 0 	mse_loss: 1.2858496904
Test -- Iteration: 0 	mse_loss: 1.2554919720
Iteration: 1 	mse_loss: 1.2543864250
Iteration: 2 	mse_loss: 1.2253611088
Iteration: 3 	mse_loss: 1.1987249851
Iteration: 4 	mse_loss: 1.1744000912
Iteration: 5 	mse_loss: 1.1522874832
Iteration: 6 	mse_loss: 1.1322715282
Iteration: 7 	mse_loss: 1.1142244339
Iteration: 8 	mse_loss: 1.0980088711
Iteration: 9 	mse_loss: 1.0834842920
Iteration: 10 	mse_loss: 1.0705088377
Iteration: 11 	mse_loss: 1.0589436293
Iteration: 12 	mse_loss: 1.0486544371
Iteration: 13 	mse_loss: 1.0395128727
Iteration: 14 	mse_loss: 1.0313990116
Iteration: 15 	mse_loss: 1.0242009163
Iteration: 16 	mse_loss: 1.0178155899
Iteration: 17 	mse_loss: 1.0121488571
Iteration: 18 	mse_loss: 1.0071156025
Iteration: 19 	mse_loss: 1.0026390553
Iteration: 20 	mse_loss: 1.0227922201
Test -- Iteration: 20 	mse_loss: 1.0207266808
Iteration: 21 	mse_loss: 1.0190967321
Iteration: 22 	mse_loss: 1.0153626204
Iteration: 23 	mse_loss: 1.0116922855
Iterati

KeyboardInterrupt: 

In [ ]:
bestmodel = np.argmin([r[2] for r in res])
components, metric_value, ls, l_cv, convergence, model = res[bestmodel]
pickle.dump(model.cpu(), open('./Wagner_sliceTCA-3-3-0.p', 'wb'))

In [ ]:
models = [r[-1].cpu() for r in res]
losses = [r[2] for r in res]

In [ ]:
pickle.dump([models, losses], open('./Wagner_sliceTCA_3-3-0_all-models.p', 'wb'))

In [ ]:
|